# Employee and Sales Data Cleaning & Analysis
This Jupyter Notebook cleans and analyzes employee and sales datasets.

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime

## Load the Datasets

In [24]:
employee_df = pd.read_csv("D:/Data Science/Data Science Bootcamp 2025/Pandas_project/employee_data_with_issues.csv")
sales_df = pd.read_csv("D:/Data Science/Data Science Bootcamp 2025/Pandas_project/sales_data_with_issues.csv")

## 1️⃣ Check for Missing Values

In [26]:
print("Missing Values in Employee Data:")
print(employee_df.isnull().sum())
print("\nMissing Values in Sales Data:")
print(sales_df.isnull().sum())

Missing Values in Employee Data:
Employee_ID           0
Name                  1
Date_of_Birth         1
Gender                0
Job                   2
Salary                1
Date_of_Joining       1
Performance_Rating    1
dtype: int64

Missing Values in Sales Data:
Sale_ID          0
Customer_Name    1
Product          1
Quantity         1
Price            0
Sale_Date        1
Region           1
Employee_ID      0
dtype: int64


## 2️⃣ Handle Missing Values

In [28]:
# Fill missing salaries with the median salary if the column exists
if 'Salary' in employee_df.columns:
    employee_df['Salary'] = employee_df['Salary'].fillna(employee_df['Salary'].median())

# Drop columns with more than 50% missing values
threshold = len(employee_df) * 0.5
employee_df.dropna(thresh=threshold, axis=1, inplace=True)

# Fill missing categorical values with mode
if "Name" in employee_df.columns:
    employee_df["Name"] = employee_df["Name"].fillna(employee_df["Name"].mode()[0])
if "Job" in employee_df.columns:
    employee_df["Job"] = employee_df["Job"].fillna(employee_df["Job"].mode()[0])
    
# Convert Date_of_Birth to datetime if not already
if "Date_of_Birth" in employee_df.columns:
    employee_df["Date_of_Birth"] = pd.to_datetime(employee_df["Date_of_Birth"], errors="coerce")
    median_date = employee_df["Date_of_Birth"].median()
    employee_df["Date_of_Birth"] = employee_df["Date_of_Birth"].fillna(median_date)
    
# Fill missing numerical values with mean (or median for Date_of_Birth)
if "Date_of_Birth" in employee_df.columns:
    employee_df["Date_of_Birth"] = employee_df["Date_of_Birth"].fillna(employee_df["Date_of_Birth"].median())

if "Performance_Rating" in employee_df.columns:
    employee_df["Performance_Rating"] = employee_df["Performance_Rating"].fillna(employee_df["Performance_Rating"].mean())


## 3️⃣ Convert Data Types

In [30]:
# Ensure Date columns are in datetime format
employee_df["Date_of_Birth"] = pd.to_datetime(employee_df["Date_of_Birth"], errors='coerce')
employee_df["Date_of_Joining"] = pd.to_datetime(employee_df["Date_of_Joining"], errors='coerce')

# Convert Salary to numeric
employee_df["Salary"] = pd.to_numeric(employee_df["Salary"], errors='coerce')

## 4️⃣ Feature Engineering

In [32]:
# Calculate Age
employee_df["Age"] = (pd.to_datetime("today") - employee_df["Date_of_Birth"]).dt.days // 365

# Create "Status" column based on Performance_Rating
status_mapping = {"Excellent": "Good", "Good": "Average", "Average": "Needs Improvement"}
if 'Performance_Rating' in employee_df.columns:
    employee_df['Status'] = employee_df['Performance_Rating'].map(status_mapping)

## 5️⃣ Data Analysis

In [34]:
# Check total sales per employee (if applicable)
if {'Employee_ID', 'Price', 'Quantity'}.issubset(sales_df.columns):
    sales_df['Total_Sales'] = sales_df['Price'] * sales_df['Quantity']
    total_sales_by_employee = sales_df.groupby('Employee_ID')['Total_Sales'].sum()
    print("Total Sales by Each Employee:")
    print(total_sales_by_employee)

# Show distribution of employee ages
if 'Age' in employee_df.columns:
    age_distribution = employee_df['Age'].value_counts().sort_index()
    print("Age Distribution:")
    print(age_distribution)

Total Sales by Each Employee:
Employee_ID
101    2400.0
102    4000.0
103    1000.0
104    -300.0
105     200.0
106       0.0
107    2400.0
Name: Total_Sales, dtype: float64
Age Distribution:
Age
24    1
29    2
31    1
32    2
34    2
36    1
38    1
Name: count, dtype: int64


## 6️⃣ Save Cleaned Data

In [36]:
employee_df.to_csv("D:/Data Science/Data Science Bootcamp 2025/Pandas_project/cleaned_employee_data.csv", index=False)
sales_df.to_csv("D:/Data Science/Data Science Bootcamp 2025/Pandas_project/cleaned_sales_data.csv", index=False)

## 7️⃣ Merge Employee & Sales Data

In [38]:
# Merge employee and sales data
if 'Employee_ID' in employee_df.columns and 'Employee_ID' in sales_df.columns:
    employee_sales = pd.merge(employee_df, sales_df, on="Employee_ID")
    employee_sales.to_csv("D:/Data Science/Data Science Bootcamp 2025/Pandas_project/employee_sales.csv", index=False)
    print("Merged employee-sales data saved successfully.")

print("Data cleaning and analysis completed successfully!")

Merged employee-sales data saved successfully.
Data cleaning and analysis completed successfully!
